# Introduction

This notebook will walk you through creating and monitoring your HITs. 

It provides methods to create HITs, pretty-print HIT and assignment status, expire/edit HITs, create qualifications, and download collected data. 

Before continuing, make sure that you have read the README and set all config fields to their desired values.

## Requirements: 

This code requires Python3 and the following packages: 
- boto3 
- beautiful soup 4

Before using, you will have to set up an authentication key to use the Amazon API and include it in a credentials file. See here: https://aws.amazon.com/developers/getting-started/python/

# Setup

Read the config file and establish a connection to MTurk.

A connection is made to production or to the sandbox based on values in the config. 

In [53]:
import datetime
import boto3
import json
import copy
import pprint
from bs4 import BeautifulSoup as bs 
from uuid import uuid4

In [54]:
# path to the config file 
CONFIG_PATH = "configNotebook.json"

# where to save downloaded results 
SAVE_PATH = "assignments_notSandbox.json" 

# Sandbox or Production? You only spend money in Production.
USING_PROD = True

In [55]:
# Safety flags that prevent you from accidentally messing up your HITs. 
# Set to False except when you are performing these specific tasks.
ALLOW_HIT_CREATION = True
ALLOW_ASSIGNMENT_ADDITION = True
ALLOW_CREATE_QUAL = False
ALLOW_UPDATE_EXPIRATION = True

In [56]:
# Read config and extract relevant settings
with open(CONFIG_PATH, 'r') as f:
    config = json.loads(f.read())
    
hit_config = config['hitCreation']

external_submit = config['advanced']['externalSubmit']
    
hit_url = hit_config['taskUrl']

if USING_PROD:
    print("USING PROD")
    endpoint_url = 'https://mturk-requester.us-east-1.amazonaws.com'
    origin="production"
else:
    print("USING SANDBOX")
    endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'
    origin="sandbox"

# If using an external link, add a querystring origin=sandbox or origin=production 
# for use in your js logic if you want. Not done for MTurk submits because it breaks the submit link
if external_submit: 
    hit_url = "%s?origin=%s" % (hit_url, origin)

if external_submit:
    print("Configuring task as external link with data submitted to: %s" % config['advanced']['externalSubmitUrl'])
else:
    print("Configuring task as an iframe within Mturk")
print("TASK URL: " + hit_url)

session = boto3.session.Session(profile_name='default')
cl = session.client('mturk', region_name='us-east-1', endpoint_url=endpoint_url)

USING PROD
Configuring task as an iframe within Mturk
TASK URL: https://smalpica.github.io/


# Make new HIT

In [57]:
# List of qualifications that you will use to filter potential workers. 
# These require that workers come from the US and have an approval rating >= 95%
# Edit this list to specify different qualifications for workers 
QUALS = [
    {
        'QualificationTypeId': '00000000000000000071',
        'Comparator': 'EqualTo',
        'LocaleValues': [{
            'Country': 'US',
        }],
    },
    {
        'QualificationTypeId': '000000000000000000L0',
        'Comparator': 'GreaterThanOrEqualTo',
        'IntegerValues': [
            50
        ],
    },
]

In [58]:
# Helpers for creating HITs. 

# generic helper that sets metadata fields based on the config file.
def create_hit(task, questionText, quals=QUALS): 
    response = cl.create_hit(
        MaxAssignments=task['numAssignments'],
        AutoApprovalDelayInSeconds=604800,
        LifetimeInSeconds=task['lifetime'],
        AssignmentDurationInSeconds=task['duration'],
        Reward=task['rewardAmount'],
        Title=task['title'],
        Keywords=task['keywords'],
        Description=task['description'],
        Question=questionText,
        QualificationRequirements=quals,
    )
    print(response)
    print("\n")

# creates a HIT in the form of an External Question inside an iFrame
def create_hit_iframe(task):
    questionText = "<ExternalQuestion xmlns=\"http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/"
    questionText += "2006-07-14/ExternalQuestion.xsd\">\n<ExternalURL>" + task['taskUrl']
    questionText += "</ExternalURL>\n  <FrameHeight>1200</FrameHeight>\n</ExternalQuestion>"
    create_hit(task, questionText)
    
# Helper to create a HIT in the form of a simple UI with a link to an external page and an
# input box for a completion code 
def create_hit_external(task):
    with open('questionform_template.xml', 'r') as myfile:
        template=myfile.read() 
    question_xml = template % (hit_config["title"], hit_config["description"], hit_url)
    create_hit(task, question_xml)

In [59]:
# Use this cell to launch your HIT! 
if ALLOW_HIT_CREATION: 
    if not (hit_config.get('variants', False) or hit_config.get('numTasks', False)): 
        raise RuntimeError("You must specify either hitCreation.numTasks or hitCreation.variants in your config.json file")
    
    hit_creation_function = create_hit_external if external_submit else create_hit_iframe
    
    if hit_config.get('numTasks', False): 
        print("creating " + str(hit_config['numTasks']) + " tasks")
        for i in range(hit_config['numTasks']):
            hit_creation_function(hit_config)
    else: 
        print("creating " + str(len(config['variants'])) + " variants")
        for var in hit_config['variants']: 
            task = copy.deepcopy(config)
            task.update(var)
            hit_creation_function(task)
else: 
    raise RuntimeError("This action is not currently enabled; set `ALLOW_HIT_CREATION` to true to proceed with this action")

creating 10 tasks
{'HIT': {'HITId': '3HEADTGN2RGK88VNHNURVVCT9EYVRK', 'HITTypeId': '34W4CI95J2PT8UOKIR12A1FST899IR', 'HITGroupId': '39UNKKTVEIWBSCFQPSY9E9PWX01GYU', 'CreationTime': datetime.datetime(2019, 5, 14, 14, 12, 38, tzinfo=tzlocal()), 'Title': 'Outdoor scenes', 'Description': 'Find the outdoor scenes in several blocks of images', 'Question': '<ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">\n<ExternalURL>https://smalpica.github.io/</ExternalURL>\n  <FrameHeight>1200</FrameHeight>\n</ExternalQuestion>', 'Keywords': 'outdoor, detection, images, 2D, outlier', 'HITStatus': 'Assignable', 'MaxAssignments': 1, 'Reward': '2.00', 'AutoApprovalDelayInSeconds': 604800, 'Expiration': datetime.datetime(2019, 5, 15, 14, 12, 38, tzinfo=tzlocal()), 'AssignmentDurationInSeconds': 720, 'QualificationRequirements': [{'QualificationTypeId': '00000000000000000071', 'Comparator': 'EqualTo', 'LocaleValues': [{'Country': 'US'}]

{'HIT': {'HITId': '3AXFSPQOYSMEENQK93GEBINGB3GJFS', 'HITTypeId': '34W4CI95J2PT8UOKIR12A1FST899IR', 'HITGroupId': '39UNKKTVEIWBSCFQPSY9E9PWX01GYU', 'CreationTime': datetime.datetime(2019, 5, 14, 14, 12, 39, tzinfo=tzlocal()), 'Title': 'Outdoor scenes', 'Description': 'Find the outdoor scenes in several blocks of images', 'Question': '<ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">\n<ExternalURL>https://smalpica.github.io/</ExternalURL>\n  <FrameHeight>1200</FrameHeight>\n</ExternalQuestion>', 'Keywords': 'outdoor, detection, images, 2D, outlier', 'HITStatus': 'Assignable', 'MaxAssignments': 1, 'Reward': '2.00', 'AutoApprovalDelayInSeconds': 604800, 'Expiration': datetime.datetime(2019, 5, 15, 14, 12, 39, tzinfo=tzlocal()), 'AssignmentDurationInSeconds': 720, 'QualificationRequirements': [{'QualificationTypeId': '00000000000000000071', 'Comparator': 'EqualTo', 'LocaleValues': [{'Country': 'US'}], 'RequiredToPrevi

# HIT monitoring helpers

Helper functions that will be useful for monitoring the status of your HIT. See next section for how to use them.

In [67]:
# Contacts MTurk API to get all assignments for a HIT
# Returns them in a list. 
def get_all_assignments(hitid): 
    assignments = []
    should_continue = True
    next_token = False
    while (should_continue): 
        args = {
            'HITId': hitid, 
            'MaxResults': 100
        }
        if (next_token): 
            args['NextToken'] = next_token
        r = cl.list_assignments_for_hit(**args)
        next_token = r.get('NextToken', False)
        assignments.extend(r["Assignments"])
        should_continue = len(r["Assignments"]) > 0
    return assignments

# Summarizes all hits in `hits` in a human-readable way. 
# Prints out the HIT Title, id, if it is expired, and how many assignments it has
# completed, pending, and left for work. 
def summarize_hits(hits): 
    print(len(hits))
    ret = ""
    for hit in hits: 
        expiration = hit['Expiration'].replace(tzinfo=None)
        is_expired = expiration < datetime.datetime.now()
        description = ("Title: {title}\n" 
        "ID: {hid}\n"
        "\tAssignments left: {left}\n"
        "\tAssignments completed: {complete}\n"
        "\tAssignments pending: {pending}\n"
        "\tExpired: {exp}\n\n").format(
            title=hit['Title'], 
            hid=hit['HITId'], 
            left=hit['NumberOfAssignmentsAvailable'], 
            complete=hit['NumberOfAssignmentsCompleted'], 
            pending=hit['NumberOfAssignmentsPending'],
            exp=str(is_expired)
        )
        ret += description
    print(ret)
    
# Prints a human-readable summary of all pending/submitted/approved assignments for all hits in `hits`
def summarize_assignments(hits):
    ret = ""
    for hit in hits: 
        hid = hit['HITId']
        title =  hit['Title']
        name = "HIT %s: %s" % (hid, title)
        ret += name + "\n"
        assignments = get_all_assignments(hid)
        if len(assignments) == 0: 
            ret += "\tNo pending/submitted/approved assignments for this HIT\n"
        for a in assignments: 
            desc = "\tAssignment {aid}\n\t\tStatus: {status}\n".format(aid=a['AssignmentId'], status=a['AssignmentStatus'])
            ret += desc
    print(ret)
    
# Refreshes data about the requested hits
def refresh_hits(): 
    global hits 
    global MAX_RESULTS
    hits = cl.list_hits(MaxResults=MAX_RESULTS)['HITs']

# HIT monitoring

In [68]:
MAX_RESULTS = 30 # set equal to the number of outstanding hits you have 

# API call to grab HIT data from MTurk 
hits = cl.list_hits(MaxResults=MAX_RESULTS)['HITs']

In [69]:
# Summarizes all outstanding HITs
refresh_hits()
summarize_hits(hits)

10
Title: Outdoor scenes
ID: 3NRZ1LDP7YUG5FKI5SF5B6PAW7GPZM
	Assignments left: 0
	Assignments completed: 0
	Assignments pending: 1
	Expired: False

Title: Outdoor scenes
ID: 3OPLMF3EU7BWX5R1ACB4Z29HHJ7NL5
	Assignments left: 0
	Assignments completed: 0
	Assignments pending: 1
	Expired: False

Title: Outdoor scenes
ID: 3AXFSPQOYSMEENQK93GEBINGB3GJFS
	Assignments left: 0
	Assignments completed: 0
	Assignments pending: 1
	Expired: False

Title: Outdoor scenes
ID: 31MBOZ6PAQFD61UALM2QJRF5IFTLCL
	Assignments left: 0
	Assignments completed: 0
	Assignments pending: 1
	Expired: False

Title: Outdoor scenes
ID: 3JY0Q5X05LUA1LWECH1TWQVCKQZGGK
	Assignments left: 0
	Assignments completed: 0
	Assignments pending: 1
	Expired: False

Title: Outdoor scenes
ID: 3HA5ODM5KCGW9NV0WCUWL5HPVGHVS8
	Assignments left: 0
	Assignments completed: 0
	Assignments pending: 1
	Expired: False

Title: Outdoor scenes
ID: 3HA5ODM5KCGW9NV0WCUWL5HPVGHSV5
	Assignments left: 0
	Assignments completed: 0
	Assignments pending: 1

In [70]:
# Summarizes assignments for all oustanding HITs 
refresh_hits()
summarize_assignments(hits)

HIT 3NRZ1LDP7YUG5FKI5SF5B6PAW7GPZM: Outdoor scenes
	No pending/submitted/approved assignments for this HIT
HIT 3OPLMF3EU7BWX5R1ACB4Z29HHJ7NL5: Outdoor scenes
	No pending/submitted/approved assignments for this HIT
HIT 3AXFSPQOYSMEENQK93GEBINGB3GJFS: Outdoor scenes
	No pending/submitted/approved assignments for this HIT
HIT 31MBOZ6PAQFD61UALM2QJRF5IFTLCL: Outdoor scenes
	No pending/submitted/approved assignments for this HIT
HIT 3JY0Q5X05LUA1LWECH1TWQVCKQZGGK: Outdoor scenes
	No pending/submitted/approved assignments for this HIT
HIT 3HA5ODM5KCGW9NV0WCUWL5HPVGHVS8: Outdoor scenes
	No pending/submitted/approved assignments for this HIT
HIT 3HA5ODM5KCGW9NV0WCUWL5HPVGHSV5: Outdoor scenes
	No pending/submitted/approved assignments for this HIT
HIT 3HEADTGN2RGK88VNHNURVVCT9EYVRK: Outdoor scenes
	No pending/submitted/approved assignments for this HIT
HIT 356ZPKYPUJ5G0PAD96DMXZG8QDCPYV: Outdoor scenes
	No pending/submitted/approved assignments for this HIT
HIT 37MQ8Z1JQGKK8I97782A1M8FF3KY2H: O

# Approve HITs

Approves all outstanding assignments for the HITs displayed above. 

In [22]:
def approve_all(hits): 
    num_approved = 0
    for hit in hits: 
        # make sure you keep getting assignments 
        assignments = get_all_assignments(hit["HITId"])
        #print(assignments)
        for a in assignments: 
            if a['AssignmentStatus'] != 'Approved':
                print("Approving assignment")
                num_approved += 1
                cl.approve_assignment(AssignmentId=a['AssignmentId'])
    print("Approved %d assignments" % num_approved)

In [23]:
refresh_hits()
approve_all(hits)

Approving assignment
Approved 1 assignments


# Update expiration or num tasks

In [50]:
# changes the expiration date on a HIT to days_from_now days in the future
def update_expiration(hitid, days_from_now): 
    if ALLOW_UPDATE_EXPIRATION: 
        days = days_from_now*datetime.timedelta(days=1)
        expire_time = datetime.datetime.now() + days

        response = cl.update_expiration_for_hit(HITId=hitid, ExpireAt=expire_time)
        print(response)
        return response
    else: 
        raise RuntimeError("This action is not currently enabled; set `ALLOW_UPDATE_EXPIRATION` to true to proceed with this action")
    
def expire_hit(hit): 
    return update_expiration(hit, -10)

In [51]:
def add_assignments(hitid, num_assignments): 
    if ALLOW_ASSIGNMENT_ADDITION: 
        response = cl.create_additional_assignments_for_hit(
            HITId=hitid,
            NumberOfAdditionalAssignments=num_assignments
        )
        print(response)
        return response
    else: 
        raise RuntimeError("This action is not currently enabled; set `ALLOW_ASSIGNMENT_ADDITION` to true to proceed with this action")

In [52]:
# Use this cell to expire a HIT 
#HIT_id_to_expire = "FILL THIS IN" 
HIT_id_to_expire ="3GKAWYFRARHD2EFMDAQNS31G9BHDPO"
expire_hit(HIT_id_to_expire)

{'ResponseMetadata': {'RequestId': 'a663daa5-db42-4b1b-84b5-2d872c3c5fe2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a663daa5-db42-4b1b-84b5-2d872c3c5fe2', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Tue, 14 May 2019 21:07:44 GMT'}, 'RetryAttempts': 0}}


{'ResponseMetadata': {'RequestId': 'a663daa5-db42-4b1b-84b5-2d872c3c5fe2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a663daa5-db42-4b1b-84b5-2d872c3c5fe2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'date': 'Tue, 14 May 2019 21:07:44 GMT'},
  'RetryAttempts': 0}}

In [ ]:
# Use this cell to add assignments to a HIT 
HIT_id_to_add_assignments = "FILL THIS IN"
num_assignments_to_add = 0
add_assignments(HIT_id_to_add_assignments, num_assignments_to_add)

# Add custom qualifications 

## Add a qualification to disqualify workers who have done work before

- uses "negative qualification" method from https://github.com/cloudyr/MturkR/wiki/qualifications-as-blocks

#### NOTE: quals are kept separate for the sandbox and prod. Make sure you are creating and assigning your quals in prod. 

### Structure of a new qualification

In [64]:
NEW_QUAL = {
    'Name': 'Auto-Granted Qualification to Prevent Retakes',
    'Keywords': 'Worked for me before',
    'Description': 'This qualification is for people who have worked for me on this task before. It is granted automatically to new workers.',
    'QualificationTypeStatus': 'Active',
    'AutoGranted': True,
    'value': 100
}

### Helpers for creating, viewing, and assigning qualifications

In [65]:
# Registers a custom qualification with MTurk 
def create_qual(new_qual):
    if ALLOW_CREATE_QUAL: 
        response = cl.create_qualification_type(**new_qual)
        print(response)
        Id = response['QualificationType']['QualificationTypeId']
        print("id", Id)
        return Id
    else: 
        raise RuntimException("This action is not currently enabled; set `ALLOW_CREATE_QUAL` to true to proceed with this action")
        
# Gets all the custom quals you have created and prints them
def list_quals(): 
    response = cl.list_qualification_types(
            Query='hasCompletedVisualGraphRecallTask',
            MustBeRequestable=False
    )
    print(response)
    
# Assigns a qualification to a worker 
def assign_qual(qual_id, worker_ids): 
    for worker in worker_ids: 
        response = cl.associate_qualification_with_worker(
                QualificationTypeId=qual_id, 
                WorkerId=worker,
                IntegerValue=1,
                SendNotification=False
        )
        print(response)
        assert response
        
# Gets the ids of all workers who worked on a particular hit 
def get_workers_for_hit(hitid): 
    a = get_all_assignments(hitid)
    workers = [a_['WorkerId'] for a_ in a]
    return workers
    
# Confirms that every worker in worker_ids has qual with qual_id
def confirm_quals(qual_id, worker_ids): 
    for w in worker_ids: 
        response = cl.get_qualification_score(
                QualificationTypeId=qual_id,
                WorkerId=w
        )
        response = response['Qualification']
        assert response['Status'] == 'Granted'
        assert response['IntegerValue'] == 1
        
# Assigns qual with `qual_id` to every worker who has completed an assignment for the hit with `hitid`
def assign_qual_for_hit(hitid, qual_id): 
    workers = get_workers_for_hit(hitid)
    print("got workers")
    assign_qual(qual_id, workers)
    print("assigned qual")
    confirm_quals(qual_id, workers)
    print("confirmed qual")

### Use the following cells to manipulate qualifications

In [66]:
# Use this cell to view the custom qualifications you have created
list_quals()

{'NumResults': 1, 'NextToken': 'p1:w9l2UKZr5Av85xagEHdbUdM5R6XCxRCMWTv+Cjd2VB9OBhTwWmR+7lAbph8rCA==', 'QualificationTypes': [{'QualificationTypeId': '3Q3I6L0BKOF89DJMPYQCYLJBC2VW5E', 'CreationTime': datetime.datetime(2018, 3, 23, 14, 19, 7, tzinfo=tzlocal()), 'Name': 'hasCompletedVisualGraphRecallTask', 'Description': 'Assigned to people who have already completed work on a visual graph recall task. A worker can only complete this type of task once.', 'Keywords': 'Already completed visual graph recall task', 'QualificationTypeStatus': 'Active', 'IsRequestable': True, 'AutoGranted': False}], 'ResponseMetadata': {'RequestId': 'f271ce5b-1893-40c3-9794-5f1599cd33f6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f271ce5b-1893-40c3-9794-5f1599cd33f6', 'content-type': 'application/x-amz-json-1.1', 'content-length': '531', 'date': 'Tue, 14 May 2019 21:17:10 GMT'}, 'RetryAttempts': 0}}


In [ ]:
# Use this cell to create a new qual 
qual_to_create = {}
create_qual(qual_to_create)

In [ ]:
# Use this cell to assign a custom qual to every worker who has done a specific HIT
hit_id = "FILL THIS IN"
qual_id_to_assign = "FILL THIS IN"
assign_qual_for_hit(hit_id, qual_id_to_assign)

# Create Compensation HIT

Mistakes happen, and sometimes they can lead to a worker who put in an honest effort being unable to complete a task and get paid. It's a good idea to compensate these workers when they reach out because it helps maintain relations with workers and is the right thing to do.

However, workers can only be paid upon completing a task. The workaround is to create a custom qualification, assign it to the worker you want to compensate, and create a no-work HIT requiring the custom qualification. This code does that.

In [ ]:
# worker_ids is str[]
# compensation is str but should match the regex ^\d*\.\d\d$ (e.g. "1.00")
# for_hit_id is str -- optional, but helpful for records
def compensate_workers(worker_ids, compensation, for_hit_id=""):
    with open('compensation.xml', 'r') as myfile:
        question_xml=myfile.read()

    keywords = 'compensation'
    description = 'Compensation for HIT'
    if for_hit_id:
        keywords += ', ' + for_hit_id
        description += ' ' + for_hit_id

    # create qual, assign to workers
    custom_qual = {
        'Name': str(uuid4()), # a qual must have a unique name
        'Keywords': keywords,
        'Description': description,
        'QualificationTypeStatus': 'Active',
        'AutoGranted': False
    }
    qual_id = create_qual(custom_qual)
    assign_qual(qual_id, worker_ids)

    # create HIT requiring qual
    task = {
        'numAssignments': len(worker_ids),
        'lifetime': 3 * 24 * 60 * 60, # 3 days
        'duration': 5 * 60, # 5 min
        'rewardAmount': compensation,
        'title': description,
        'keywords': keywords,
        'description': description,
    }
    quals = [{
        'QualificationTypeId': qual_id,
        'Comparator': 'Exists',
        'ActionsGuarded': 'DiscoverPreviewAndAccept'
    }]
    create_hit(task, question_xml, quals)

In [ ]:
worker_ids = [] # worker_id strings in a list
compensation = "0.00" # change to the amount of dollars you want to give
for_hit_id = "" # hit_id string (what you are compensating for)compensate_workers(worker_ids, compensation, for_hit_id)

# Download data

Helper to download data from MTurk 

In [24]:
def pretty_print(obj):
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(obj)
    pp = None

# Downloads all the assignments completed for `hits` as a list of dictionaries. 
# If a download_path is given, also saves that data as json 
def get_assignment_content(hits, download_path="", should_print=False): 
    all_responses = []
    for hit in hits: 
        hitid = hit['HITId']
        assignments = get_all_assignments(hitid)
        for a in assignments:
            a_xml = a['Answer']
            #print(a_xml)
            soup = bs(a_xml, "html.parser")
            answers = soup.find_all("answer")
            #print(answers)
            results = {'HITId': a['HITId'], 'AssignmentId': a['AssignmentId'], 'WorkerId': a['WorkerId']}
            for ans in answers: 
                identifier = ans.find('questionidentifier').string
                answer = ans.find('freetext').string
                try: 
                    results[identifier] = json.loads(answer)
                except:
                    results[identifier] = answer
            all_responses.append(results)
    if should_print: 
        pretty_print(all_responses)
    if download_path: 
        with open(download_path, 'w') as outfile: 
            json.dump(all_responses, outfile)
    return all_responses
            

In [25]:
# Use this cell to download data
responses = get_assignment_content(hits, download_path=SAVE_PATH, should_print=True)
pass

[   {   'AssignmentId': '3RXPCZQMQSZGJQ45L0L6LCIWPRS1G1',
        'HITId': '39XCQ6V3K0SAOVIOJ89VORM71PT562',
        'WorkerId': 'A2RDZ478HBSFYG',
        'feedback': None,
        'results': {   'assignmentId': '3RXPCZQMQSZGJQ45L0L6LCIWPRS1G1',
                       'hitId': '39XCQ6V3K0SAOVIOJ89VORM71PT562',
                       'results': {   'inputs': {   'bucketNum': 0,
                                                    'params': {   'ENABLE_MSEC_SET_URL': False,
                                                                  'GIVE_FEEDBACK': False,
                                                                  'IMG_HEIGHT': 700,
                                                                  'IMG_WIDTH': 1000,
                                                                  'MAX_INCORRECT_SENTINELS_ALLOWED_TUTORIAL': 3,
                                                                  'MAX_INVALID_ALLOWED_TUTORIAL': 3,
                                                  